![](NEURAL_NETWORKS.jpg)

# Simple MNIST
## Done By : Alireza Khodakarami

### Imports

We need pickle to read pre-processed data from hard drive, and cv2 to read and tweak test data at the end. The rest is tensor flow and possible needs for defining our neural network model.

In [22]:
import tensorflow as tf
import pickle
import cv2

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, LeakyReLU

### Read Data

We read binary pre-processed data from hard drive and assign them to X and y respectedly.

In [52]:
pickle_in = open('./Data/PetImages/X.pickle', 'rb')
X = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open('./Data/PetImages/y.pickle', 'rb')
y = pickle.load(pickle_in)
pickle_in.close()

### Normalize

We still need to normalize the input data to make them sit between 0 - 1. We could have used the utils function from tensor flow but wanted to show you another way of doing it on images.

In [53]:
X = X / 255.0

### Print The Shape

Always a good idea of printing the shape before model definition. We need to introduce the shape in first neural layer (we can ignore it but then we could not use model.summary). Knowing the shape can help putting the proper code on the first layer within model definition. The first element is the number of data. Tesorflow don't care about the number of data but the rest of the shape is really important else, it would fail accepting the data as inputs.

In [73]:
print(X.shape)

(24946, 150, 150, 1)


### Model Definition

Finally we define a convolutional neural network. After conv2D layers, we need a flatten layer. The last layer is a single output node. We don't need more than one because our classification is a binary format, we either have dog or cat (0 or 1).

In [54]:
model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
#model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.2))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
#model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.2))

model.add(Flatten())


model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 148, 148, 256)     2560      
_________________________________________________________________
activation_10 (Activation)   (None, 148, 148, 256)     0         
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 74, 74, 256)       0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 72, 72, 256)       590080    
_________________________________________________________________
activation_11 (Activation)   (None, 72, 72, 256)       0         
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 36, 36, 256)       0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 331776)            0         
__________

### Define Fitting Params

This is one of my personal habbits. I prefer to have a dedicated section for parameters I use on model training. It makes it easier for me to read and later on, tweak these params if needed.

In [55]:
BATCH_SIZE = 32
EPOCHS = 30
VALIDAITON_SPLIT = 0.1

### Training Model

Finally we send in the inputs and labels for the model to train itself on as many epochs as we think we need.

In [56]:
model.fit(X, y,
         batch_size = BATCH_SIZE,
         epochs = EPOCHS,
         validation_split = VALIDAITON_SPLIT)

Train on 22451 samples, validate on 2495 samples
Epoch 1/30
22451/22451 [==============================] - 64s 3ms/sample - loss: 0.6280 - acc: 0.6461 - val_loss: 0.7818 - val_acc: 0.5407
Epoch 2/30
22451/22451 [==============================] - 63s 3ms/sample - loss: 0.5248 - acc: 0.7401 - val_loss: 0.6284 - val_acc: 0.6890
Epoch 3/30
22451/22451 [==============================] - 65s 3ms/sample - loss: 0.4701 - acc: 0.7750 - val_loss: 0.4855 - val_acc: 0.7760
Epoch 4/30
22451/22451 [==============================] - 65s 3ms/sample - loss: 0.4263 - acc: 0.8040 - val_loss: 0.5426 - val_acc: 0.7507
Epoch 5/30
22451/22451 [==============================] - 64s 3ms/sample - loss: 0.3987 - acc: 0.8176 - val_loss: 0.5092 - val_acc: 0.7543
Epoch 6/30
22451/22451 [==============================] - 63s 3ms/sample - loss: 0.3690 - acc: 0.8373 - val_loss: 0.5007 - val_acc: 0.7844
Epoch 7/30
22451/22451 [==============================] - 62s 3ms/sample - loss: 0.3381 - acc: 0.8543 - val_loss: 0.6

### Save Model

At the end of each training session we save the model so that we won't loose trained model by any accident.

In [57]:
model.save('./Models/03 - Convolutional.model')

### Load Model

In one single training session you won't need this. But if you are planning on training your model for many days, there is a high chance that you would want to turn off the pc, and next day continue the job. Having this section lets you load the previously saved model and save yourself the troble of reteaching the model from scratch.

In [35]:
model = load_model('./Models/03 - Convolutional.model')

### Test Pre-Processing

We need to mimic the pre-processing functionality for any test image we want to use after training our model.

Here we define a function, we send in the path of the image in and we get the reshaped ready to use data back.

In [61]:
categories = ['Dog', 'Cat']

def prepare(filepath):
    img_size = 150
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (img_size, img_size))
    return new_array.reshape(-1, img_size, img_size, 1)

### Test The Prediction

Never trust the accuracy and loss value you get on the training session being it on the training data or on the validation data. You may get a good and high result there but with producing new images by yourself (something that the network never saw) you may be suprised how poor your network's prediction can become. The reverse can be true too. The result on training may not be that high and nice but the network would predict your sample tests very well. Long story short, after each training session, test your network with unseen data.

In [72]:
prediction = model.predict([prepare('./Data/Test/a01.jpg')])
print('a01 - should be human, prediction :',categories[int(prediction[0][0])])

prediction = model.predict([prepare('./Data/Test/c01.jpg')])
print('c01 - should be cat, prediction :',categories[int(prediction[0][0])])

prediction = model.predict([prepare('./Data/Test/c02.jpeg')])
print('c02 - should be cat, prediction :',categories[int(prediction[0][0])])

prediction = model.predict([prepare('./Data/Test/c03.jpg')])
print('c03 - should be cat, prediction :',categories[int(prediction[0][0])])

prediction = model.predict([prepare('./Data/Test/d01.png')])
print('d01 - should be dog, prediction :',categories[int(prediction[0][0])])

prediction = model.predict([prepare('./Data/Test/d02.jpg')])
print('d02 - should be dog, prediction :',categories[int(prediction[0][0])])

prediction = model.predict([prepare('./Data/Test/d03.jpg')])
print('d03 - should be dog, prediction :',categories[int(prediction[0][0])])

a01 - should be human, prediction : Dog
c01 - should be cat, prediction : Cat
c02 - should be cat, prediction : Cat
c03 - should be cat, prediction : Cat
d01 - should be dog, prediction : Dog
d02 - should be dog, prediction : Dog
d03 - should be dog, prediction : Dog
